In [1]:
%matplotlib inline

In [2]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score

import numpy as np
import PIL
import random
from IPython.core.display import Image, display

import matplotlib.pyplot as plt

In [3]:
DEVICE_ID = 0
DEVICE = torch.device('cuda:%d' % DEVICE_ID)
torch.cuda.set_device(DEVICE_ID)

In [4]:
#Для запуска без GPU раскомментировать и закоментировать код выше
#DEVICE = torch.device('cpu')

In [5]:
np.random.seed(100500)

def data2image(data):
    res = np.transpose(np.reshape(data ,(3, 32,32)), (1,2,0))
    return PIL.Image.fromarray(np.uint8(res))

def imshow(img):
    if isinstance(img, torch.Tensor): img = img.numpy().astype('uint8')
    plt.imshow(np.transpose(img, (1, 2, 0)))
    
def prediction2classes(output_var):
    _, predicted = torch.max(output_var.data, 1)
    predicted.squeeze_()
    classes = predicted.tolist()
    return classes

def make_solution_pytorch(net, input_tensor, a_batch_size):
    res = []
    net = net.eval()
    cur_pos = 0
    while cur_pos <= len(input_tensor):
        outputs = net(input_tensor[cur_pos:cur_pos+a_batch_size])
        res += prediction2classes(outputs)
        cur_pos += a_batch_size
    return res

In [6]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

class CifarDataset(Dataset):
    def __init__(self, input_path, is_train = True, transform = None):
                        
        data = np.load(input_path)
        if is_train: 
            self.Y, self.X = np.hsplit(data, [1]) 
            self.Y = [item[0] for item in self.Y]
        else: 
            self.X = data
            self.Y = None
            
        self.X = self.X.reshape((self.X.shape[0], 3, 32, 32))
        self.X = self.X.transpose((0, 2, 3, 1)) #приводим к виду (N, H, W, C)
        self.X = [Image.fromarray(img) for img in self.X]
                
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        
        sample = self.X[idx]

        if self.transform: sample = self.transform(sample)

        if self.Y is None: return sample
        else: return (sample, self.Y[idx])

In [7]:
train_path = 'homework_4.train.npy'
test_path  = 'homework_4_no_classes.test.npy'

In [8]:
np_mean = np.mean([item[0].numpy()
                   for item in CifarDataset(train_path, transform=transforms.ToTensor())], axis=(0,2,3))

np_std  = np.std([item[0].numpy()
                 for item in CifarDataset(train_path, transform=transforms.ToTensor())], axis=(0,2,3))

In [9]:
cifar_transform_norm = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(torch.FloatTensor(np_mean), torch.FloatTensor(np_std))
]
)

cifar_test_transform_norm = transforms.Compose([    
    transforms.ToTensor(),
    transforms.Normalize(torch.FloatTensor(np_mean), torch.FloatTensor(np_std))
]
)

In [10]:
dataset_train_norm = CifarDataset(train_path, transform=cifar_transform_norm)
dataloader_train_norm = DataLoader(dataset_train_norm, batch_size=128,
                        shuffle=True, num_workers=4)

dataset_test_norm = CifarDataset(test_path, is_train=False, transform=cifar_test_transform_norm)
dataloader_test_norm = DataLoader(dataset_test_norm, batch_size=128,
                        shuffle=False, num_workers=1)


def train_network(a_net, 
                  a_device,
                  dataloader_train_norm = dataloader_train_norm,
                  a_epochs = 70,
                  a_batch_size = 128,
                  a_lr = 0.1):
    
    train_acc = []
    net = a_net.to(a_device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(a_net.parameters(), lr = a_lr, weight_decay = 0.0001, momentum = 0.9)

    
    for epoch in range(a_epochs):  # loop over the dataset multiple times
        if epoch == 30:
            optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr/10, weight_decay=0.0001, momentum=0.9) 
        elif epoch == 50:
            optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr/100, weight_decay=0.0001, momentum=0.9) 
        
        net = net.train()        
        epoch_accuracy = 0.0
        epoch_iters = 0
        for item in dataloader_train_norm:            
            epoch_iters += 1

            inputs = item[0].to(a_device)
            labels = item[1].long().to(a_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_accuracy += accuracy_score(labels, prediction2classes(outputs))

        epoch_accuracy /= epoch_iters
        train_acc.append(epoch_accuracy)
        
        print("Epoch ", epoch, round(train_acc[-1], 4))

    print('Finished Training')
    
    plt.plot(train_acc, label='Train')
    plt.legend()
    plt.grid()

In [11]:
def conv3x3(a_in_planes, a_out_planes, a_stride=1):
    """
    Основной строительный блок конволюций для ResNet
    Включает в себя padding=1 - чтобы размерность сохранялась после его применения
    """
    return nn.Conv2d(a_in_planes, a_out_planes,  stride = a_stride,
                     kernel_size = 3, padding = 1, bias = False)

def x_downsample(a_in_channels, a_out_channels, a_stride = 2):
    return nn.Conv2d(a_in_channels, a_out_channels,  stride = a_stride,
                     kernel_size = 1, bias = False)

In [12]:
class CifarResidualBlock(nn.Module):
    def __init__(self, a_in_channels, make_downsample=False, use_skip_connection=True):
        super(CifarResidualBlock, self).__init__()
        self.use_skip_connection = use_skip_connection
        self.make_downsample = make_downsample
        
        if self.make_downsample: coef = 2
        else: coef = 1
        
        self.d  = x_downsample(a_in_channels, a_in_channels * coef)
        
        self.c1 = conv3x3(a_in_channels, a_in_channels * coef, a_stride = coef)
        self.n1 = nn.BatchNorm2d(a_in_channels * coef, eps=1e-5, momentum=0.1, affine=False)
        self.r1 = nn.ReLU()
        
        self.c2 = conv3x3(a_in_channels * coef, a_in_channels * coef)
        self.n2 = nn.BatchNorm2d(a_in_channels * coef, eps=1e-5, momentum=0.1, affine=False)
        self.r2 = nn.ReLU()
        
        self.r3 = nn.ReLU()
            
    def forward(self, x):   
        if self.use_skip_connection:
            x_prev = x
        
        x = self.c1 (x)
        x = self.n1 (x)
        x = self.r1 (x)
        
        x = self.c2 (x)
        x = self.n2 (x)
        x = self.r2 (x)
        
        if self.use_skip_connection:
            if self.make_downsample:
                x += self.d (x_prev)
                
            else:
                x += x_prev
            
        x = self.r3 (x)
        
        return x

In [13]:
class CifarResNet(nn.Module):
    
    def __init__(self):
        super(CifarResNet, self).__init__()
        
        self.features = nn.Sequential()
        
        self.features.add_module('conv_block1', conv3x3(3, 8, a_stride = 1))
        self.features.add_module('batchnrm_block1', nn.BatchNorm2d(8, eps=1e-5, momentum=0.1, affine=False))
        self.features.add_module('relu_block1', nn.ReLU())
        
        self.features.add_module('conv_block2', conv3x3(8, 32, a_stride = 1))
        self.features.add_module('batchnrm_block2', nn.BatchNorm2d(32, eps=1e-5, momentum=0.1, affine=False))
        self.features.add_module('relu_block2', nn.ReLU())
        
        self.features.add_module('conv_block3', conv3x3(32, 64, a_stride = 1))
        self.features.add_module('batchnrm_block3', nn.BatchNorm2d(64, eps=1e-5, momentum=0.1, affine=False))
        self.features.add_module('relu_block3', nn.ReLU())
        
        self.features.add_module('drop_block', nn.Dropout(0.02))
        
        self.features.add_module('res_block1', CifarResidualBlock(64, make_downsample=False))
        self.features.add_module('res_block2', CifarResidualBlock(64, make_downsample=False))
        self.features.add_module('res_block3', CifarResidualBlock(64, make_downsample=True))
        self.features.add_module('res_block4', CifarResidualBlock(128, make_downsample=False, use_skip_connection=False))
        self.features.add_module('res_block5', CifarResidualBlock(128, make_downsample=False))
        self.features.add_module('res_block6', CifarResidualBlock(128, make_downsample=False))
        self.features.add_module('res_block7', CifarResidualBlock(128, make_downsample=True))
        self.features.add_module('res_block8', CifarResidualBlock(256, make_downsample=False, use_skip_connection=False))
        self.features.add_module('res_block9', CifarResidualBlock(256, make_downsample=False))
        self.features.add_module('res_block10', CifarResidualBlock(256, make_downsample=False))
        self.features.add_module('res_block11', CifarResidualBlock(256, make_downsample=False))
        self.features.add_module('res_block12', CifarResidualBlock(256, make_downsample=False))
        self.features.add_module('res_block13', CifarResidualBlock(256, make_downsample=True))
        
        self.global_avg_pooling = nn.AvgPool2d(kernel_size=2)
        self.fc_classifier = nn.Linear(512*2*2, 100)
        
    def forward(self, x):
        x = self.features(x)
        x = self.global_avg_pooling(x)        
        x = x.view((x.size()[0], -1))        
        x = self.fc_classifier(x)        
        return x

In [14]:
resnet = CifarResNet()
%time train_network(resnet, torch.device(DEVICE))

Epoch  0 0.0256
Epoch  1 0.0691
Epoch  2 0.1109
Epoch  3 0.1559
Epoch  4 0.2047
Epoch  5 0.2607
Epoch  6 0.3131
Epoch  7 0.3606
Epoch  8 0.4031
Epoch  9 0.4341
Epoch  10 0.4629
Epoch  11 0.4924
Epoch  12 0.513
Epoch  13 0.5323
Epoch  14 0.551
Epoch  15 0.5712
Epoch  16 0.5848
Epoch  17 0.5982
Epoch  18 0.6118
Epoch  19 0.6186
Epoch  20 0.6323
Epoch  21 0.6414
Epoch  22 0.6521
Epoch  23 0.6576
Epoch  24 0.6698
Epoch  25 0.6801
Epoch  26 0.6862
Epoch  27 0.6947
Epoch  28 0.7039
Epoch  29 0.7057
Epoch  30 0.7992
Epoch  31 0.8272
Epoch  32 0.839
Epoch  33 0.8453


Process Process-137:
Process Process-140:
Process Process-139:
Process Process-138:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/r.yunusov/yes/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/r.yunusov/yes/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/r.yunusov/yes/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r.yunusov/yes/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/r.yunusov/yes/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/r.yunusov/yes/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/r.yunusov/yes/lib/python3.7/site-packages/torch/utils/da

KeyboardInterrupt: 

# Важно переключить сеть в режим eval - иначе dropout будет работать некорректно

In [18]:
def make_solution(a_net, a_device):
    res = []
    net = a_net.eval()
    for item in dataloader_test_norm:
        inputs = item.to(a_device)
        outputs = net(inputs) 

        res += prediction2classes(outputs)
    return res

In [19]:
# my_solution = make_solution(dense_net, DEVICE)
my_solution = make_solution(resnet, DEVICE)

In [20]:
with open('my_solution.csv', 'w') as fout:
    print('Id', 'Prediction', sep=',', file=fout)
    for i, prediction in enumerate(my_solution):
        print(i, prediction, sep=',', file=fout)